**This notebook shows how transfer learning can be used to improve the global temperature prediction. <br/>**

By Weiwei Zhan


We train NN models using two different settings, i.e., w/o and with transfer learning, and compare how the transfer learning affects our prediction:

**w/o transfer learning:** train a NN model from scratch <br/>
**with transfer learning:** initialize the NN model with pre-trained weights that are obtained from other scenarios


In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from glob import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential
from utils import * 
tf.random.set_seed(2022)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

plt.rcParams['savefig.dpi'] = 400
plt.rcParams['font.size'] = 13
plt.rcParams["legend.frameon"] = False

In [2]:
cwd = os.getcwd()

train_path = os.path.join(cwd,'Data','train_val')
test_path  = os.path.join(cwd,'Data','test')

make_dir(train_path)
make_dir(test_path)

### 1. data preprocssing: prepare data for training & test sets

#### import data as training & test sets

training set: the `ssp126` scenarios <br/>
test set: the `ssp245` scenario

In [3]:
# Training set
scenario_list  = ["historical","ssp585","ssp370", "ssp126","hist-aer","hist-GHG"]
train_files    = ["ssp370"]
X_train_xr, _  = prepare_predictor(train_files,train_path)
y_train_xr, _  = prepare_predictand(train_files,train_path)

# Test set
X_test_xr, _ = prepare_predictor('ssp245', data_path=test_path,time_reindex=False)
y_test_xr, _ = prepare_predictand('ssp245',data_path=test_path,time_reindex=False)

#### select relevant variables

predictors: CO2 & CH4 <br/>
predictand: tas

In [4]:
X_train_df = pd.DataFrame({"CO2": X_train_xr["CO2"].data,
                           "CH4": X_train_xr["CH4"].data
                          }, index=X_train_xr["CO2"].coords['time'].data)

X_test_df  = pd.DataFrame({"CO2": X_test_xr["CO2"].data,
                           "CH4": X_test_xr["CH4"].data
                          }, index=X_test_xr["CO2"].coords['time'].data)


y_train_df = y_train_xr["tas"].stack(dim=["latitude", "longitude"])
y_train_df = pd.DataFrame(y_train_df.to_pandas())

#### Data standardization

In [5]:
# Standardization
mean, std = X_train_df.mean(), X_train_df.std()

X_train_df   = (X_train_df - mean)/std
X_test_df    = (X_test_df - mean)/std

X_train = X_train_df.to_numpy()
y_train = y_train_df.to_numpy()
X_test = X_test_df.to_numpy()

print(X_train.shape,y_train.shape,X_test.shape)

(86, 2) (86, 13824) (86, 2)


### 2. train & evaluate the vanilla NN w/o transfer learning

#### define & train the model

In [6]:
# set hyperparameters
n_neuron       = 64
activation     = 'relu'
num_epochs     = 200
learning_rate  = 1e-4
minibatch_size = 64
model_num      = 1

In [7]:
model_vanilla = Sequential()

model_vanilla.add(Dense(n_neuron,  activation=activation,input_shape=(X_train.shape[1],))) #  the 1st hidden layer 
model_vanilla.add(Dense(n_neuron,  activation=activation)) # the 2nd hidden layer
model_vanilla.add(Dense(n_neuron,  activation=activation)) # the 3rd hidden layer
model_vanilla.add(Dense(y_train.shape[1],  activation='linear')) # the output layer


model_vanilla.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))

2022-11-16 17:38:34.459501: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
model_vanilla.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                192       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 13824)             898560    
                                                                 
Total params: 907,072
Trainable params: 907,072
Non-trainable params: 0
_________________________________________________________________


In [9]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)


history = model_vanilla.fit(X_train, y_train, 
                            batch_size      = minibatch_size,
                            epochs          = num_epochs,
                            validation_split= 0.2, 
                            verbose         = 1,
                            callbacks       = [early_stop])



Epoch 1/200
2/2 [==============================] - 0s 81ms/step - loss: 6.1061 - val_loss: 17.0325
Epoch 2/200
2/2 [==============================] - 0s 15ms/step - loss: 6.1033 - val_loss: 17.0244
Epoch 3/200
2/2 [==============================] - 0s 16ms/step - loss: 6.1005 - val_loss: 17.0160
Epoch 4/200
2/2 [==============================] - 0s 16ms/step - loss: 6.0974 - val_loss: 17.0075
Epoch 5/200
2/2 [==============================] - 0s 16ms/step - loss: 6.0944 - val_loss: 16.9987
Epoch 6/200
2/2 [==============================] - 0s 16ms/step - loss: 6.0912 - val_loss: 16.9893
Epoch 7/200
2/2 [==============================] - 0s 18ms/step - loss: 6.0879 - val_loss: 16.9799
Epoch 8/200
2/2 [==============================] - 0s 18ms/step - loss: 6.0846 - val_loss: 16.9702
Epoch 9/200
2/2 [==============================] - 0s 17ms/step - loss: 6.0811 - val_loss: 16.9600
Epoch 10/200
2/2 [==============================] - 0s 16ms/step - loss: 6.0775 - val_loss: 16.9491
Epoch 11/

2/2 [==============================] - 0s 14ms/step - loss: 4.7581 - val_loss: 12.5795
Epoch 84/200
2/2 [==============================] - 0s 16ms/step - loss: 4.7192 - val_loss: 12.4411
Epoch 85/200
2/2 [==============================] - 0s 15ms/step - loss: 4.6806 - val_loss: 12.2957
Epoch 86/200
2/2 [==============================] - 0s 15ms/step - loss: 4.6411 - val_loss: 12.1499
Epoch 87/200
2/2 [==============================] - 0s 15ms/step - loss: 4.6009 - val_loss: 12.0048
Epoch 88/200
2/2 [==============================] - 0s 15ms/step - loss: 4.5605 - val_loss: 11.8556
Epoch 89/200
2/2 [==============================] - 0s 17ms/step - loss: 4.5195 - val_loss: 11.7025
Epoch 90/200
2/2 [==============================] - 0s 16ms/step - loss: 4.4777 - val_loss: 11.5494
Epoch 91/200
2/2 [==============================] - 0s 16ms/step - loss: 4.4358 - val_loss: 11.4001
Epoch 92/200
2/2 [==============================] - 0s 16ms/step - loss: 4.3933 - val_loss: 11.2532
Epoch 93/200


2/2 [==============================] - 0s 14ms/step - loss: 1.8118 - val_loss: 1.6594
Epoch 166/200
2/2 [==============================] - 0s 14ms/step - loss: 1.7849 - val_loss: 1.6010
Epoch 167/200
2/2 [==============================] - 0s 14ms/step - loss: 1.7576 - val_loss: 1.5455
Epoch 168/200
2/2 [==============================] - 0s 14ms/step - loss: 1.7309 - val_loss: 1.4954
Epoch 169/200
2/2 [==============================] - 0s 14ms/step - loss: 1.7036 - val_loss: 1.4512
Epoch 170/200
2/2 [==============================] - 0s 14ms/step - loss: 1.6782 - val_loss: 1.4116
Epoch 171/200
2/2 [==============================] - 0s 14ms/step - loss: 1.6535 - val_loss: 1.3753
Epoch 172/200
2/2 [==============================] - 0s 15ms/step - loss: 1.6292 - val_loss: 1.3406
Epoch 173/200
2/2 [==============================] - 0s 14ms/step - loss: 1.6048 - val_loss: 1.3085
Epoch 174/200
2/2 [==============================] - 0s 14ms/step - loss: 1.5800 - val_loss: 1.2800
Epoch 175/200


#### evaluate the performance of the vanilla NN (w/o transfer learning)

In [10]:
y_pre_vanilla = model_vanilla.predict(X_test)
y_pre_vanilla = y_pre_vanilla.reshape(y_pre_vanilla.shape[0], 96, 144)

y_pre_vanilla = xr.Dataset(coords={'time': X_test_xr.time.values, 
                               'latitude': X_test_xr.latitude.values, 
                               'longitude': X_test_xr.longitude.values},
                           data_vars=dict(tas=(['time', 'latitude', 'longitude'], y_pre_vanilla)))

3/3 [==============================] - 0s 1ms/step


In [11]:
import matplotlib.colors as colors
fig, axes = plt.subplots(figsize=(15,12),ncols=2,nrows=3)

yrs = [2030, 2050, 2100]
vmin, vmax    = -2, 5
cmap = 'RdBu_r'
divnorm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0., vmax=vmax)

y_pre_vanilla.tas.sel(time=yrs[0]).plot(ax=axes[0,0], cmap=cmap,norm=divnorm, 
                                          cbar_kwargs={"label":"temperature (°C)"})
y_test_xr.tas.sel(time=yrs[0]).plot(ax=axes[0,1], cmap=cmap,norm=divnorm, 
                                    cbar_kwargs={"label":"temperature (°C)"})

y_pre_vanilla.tas.sel(time=yrs[1]).plot(ax=axes[1,0], cmap=cmap,norm=divnorm, 
                                          cbar_kwargs={"label":"temperature (°C)"})
y_test_xr.tas.sel(time=yrs[1]).plot(ax=axes[1,1], cmap=cmap,norm=divnorm, 
                                    cbar_kwargs={"label":"temperature (°C)"})

y_pre_vanilla.tas.sel(time=yrs[2]).plot(ax=axes[2,0], cmap=cmap,norm=divnorm, 
                                          cbar_kwargs={"label":"temperature (°C)"})
y_test_xr.tas.sel(time=yrs[2]).plot(ax=axes[2,1], cmap=cmap,norm=divnorm, 
                                    cbar_kwargs={"label":"temperature (°C)"})


for i, ax in enumerate(axes.flat):
    # left column: model prediction
    if i % 2 == 0:
        ax.set_title(f'tas model prediction (year = {yrs[i//2]})',fontweight='bold')
    # right column: truth tas from ssp245 simulations
    else:
        ax.set_title(f'tas truth (year = {yrs[i//2]})',fontweight='bold')
plt.tight_layout()

### 3. train NN with transfer learning

#### import a pre-trained model

The pre-trained model has the same structure but it is trained over more scenarios (`historical,ssp585,ssp126,hist-aer,hist-GHG)`

**There are two options:** we either can directly use the prepared pre-trained model by setting `use_existed_pretrain_model = True`, or we can train the pre-trained model from scratch by setting `use_existed_pretrain_model = False`.

In [12]:
use_existed_pretrain_model = False

if use_existed_pretrain_model:
    model_path = os.path.join(cwd,'saved_model')
    model_pretrain = load_model(os.path.join(model_path,'pretrained_model.h5'))
    
else:
    # Training set
    pretrain_files    = ["historical","ssp585","ssp126","hist-aer","hist-GHG"]  # "ssp370",  
    X_pretrain_xr, _  = prepare_predictor(pretrain_files,train_path)
    y_pretrain_xr, _  = prepare_predictand(pretrain_files,train_path)

    X_pretrain_df = pd.DataFrame({"CO2": X_pretrain_xr["CO2"].data,
                                  "CH4": X_pretrain_xr["CH4"].data
                              }, index=X_pretrain_xr["CO2"].coords['time'].data)
    y_pretrain_df = y_pretrain_xr["tas"].stack(dim=["latitude", "longitude"])
    y_pretrain_df = pd.DataFrame(y_pretrain_df.to_pandas())

    # Standardization
    X_pretrain_df   = (X_pretrain_df - mean)/std
    X_pretrain = X_pretrain_df.to_numpy()
    y_pretrain = y_pretrain_df.to_numpy()
    
    model_pretrain = Sequential()
    layer1=Dense(n_neuron,  activation=activation,input_shape=(X_train.shape[1],),name="layer1")
    model_pretrain.add(layer1) #  the 1st hidden layer 
    layer2 = Dense(n_neuron,  activation=activation,name="layer2")
    model_pretrain.add(layer2) # the 2nd hidden layer
    layer3 = Dense(n_neuron,  activation=activation,name="layer3")
    model_pretrain.add(layer3) # the 3rd hidden layer
    layer_final = Dense(y_train.shape[1],  activation='linear',name="layerfinal")
    model_pretrain.add(layer_final) # the output layer
    model_pretrain.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
    
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
    history = model_pretrain.fit(X_pretrain, y_pretrain, 
                                batch_size      = minibatch_size,
                                epochs          = num_epochs,
                                validation_split= 0.2, 
                                verbose         = 0,
                                callbacks       = [early_stop])

TypeError: Sequential.add() got an unexpected keyword argument 'name'

#### train a NN model based on the previous pre-trained model 

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

model_pretrain.get_layer("layer1").trainable = True
model_pretrain.get_layer("layer2").trainable = True
model_pretrain.get_layer("layer3").trainable = True
model_pretrain.get_layer("layerfinal").trainable = True
    
    
history = model_pretrain.fit(X_train, y_train, 
                    batch_size      = minibatch_size,
                    epochs          = num_epochs,
                    validation_split= 0.2, 
                    verbose         = 1,
                    callbacks       = [early_stop])



In [ ]:
y_pre_tl = model_pretrain.predict(X_test)
y_pre_tl = y_pre_tl.reshape(y_pre_tl.shape[0], 96, 144)

y_pre_tl = xr.Dataset(coords={'time': X_test_xr.time.values, 
                               'latitude': X_test_xr.latitude.values, 
                               'longitude': X_test_xr.longitude.values},
                       data_vars=dict(tas=(['time', 'latitude', 'longitude'], y_pre_tl)))

### compare the prediction with v.s. w/o transfer learning

In [ ]:
fig, axes = plt.subplots(figsize=(16,10),ncols=3,nrows=3)

yrs = [2030, 2050, 2100]
vmin, vmax    = -2, 5
cmap = 'RdBu_r'
divnorm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0., vmax=vmax)

y_pre_vanilla.tas.sel(time=yrs[0]).plot(ax=axes[0,0], cmap=cmap,norm=divnorm, 
                                          cbar_kwargs={"label":"temperature (°C)"})
y_pre_tl.tas.sel(time=yrs[0]).plot(ax=axes[0,1], cmap=cmap,norm=divnorm, 
                                          cbar_kwargs={"label":"temperature (°C)"})
y_test_xr.tas.sel(time=yrs[0]).plot(ax=axes[0,2], cmap=cmap,norm=divnorm, 
                                    cbar_kwargs={"label":"temperature (°C)"})

y_pre_vanilla.tas.sel(time=yrs[1]).plot(ax=axes[1,0], cmap=cmap,norm=divnorm, 
                                          cbar_kwargs={"label":"temperature (°C)"})
y_pre_tl.tas.sel(time=yrs[1]).plot(ax=axes[1,1], cmap=cmap,norm=divnorm, 
                                          cbar_kwargs={"label":"temperature (°C)"})
y_test_xr.tas.sel(time=yrs[1]).plot(ax=axes[1,2], cmap=cmap,norm=divnorm, 
                                    cbar_kwargs={"label":"temperature (°C)"})

y_pre_vanilla.tas.sel(time=yrs[2]).plot(ax=axes[2,0], cmap=cmap,norm=divnorm, 
                                          cbar_kwargs={"label":"temperature (°C)"})
y_pre_tl.tas.sel(time=yrs[2]).plot(ax=axes[2,1], cmap=cmap,norm=divnorm, 
                                          cbar_kwargs={"label":"temperature (°C)"})
y_test_xr.tas.sel(time=yrs[2]).plot(ax=axes[2,2], cmap=cmap,norm=divnorm, 
                                    cbar_kwargs={"label":"temperature (°C)"})


for i, ax in enumerate(axes.flat):
    # left column: model prediction without transfer learning
    if i % 3 == 0:
        ax.set_title(f'w/o transfer learning (year = {yrs[i//3]})',fontweight='bold')
        
    # middle column: model prediction with transfer learning
    elif i % 3 == 1:
        ax.set_title(f'with transfer learning (year = {yrs[i//3]})',fontweight='bold')
    
    # right column: truth tas from ssp245 simulations
    else:
        ax.set_title(f'tas truth (year = {yrs[i//3]})',fontweight='bold')


plt.tight_layout()